In [1]:
import requests
import numpy as np
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
import tensorflow as tf
import random
import os

In [3]:
# semillas
SEED = 69
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

In [4]:
import yfinance as yf
import pandas as pd

# Definir el ticker
ticker = "NVDA"

# Descargar datos históricos
nvda = yf.download(ticker, start="2010-01-01", end="2025-10-16", interval="1d")

# Ver las primeras filas
# print(nvda.head())

# Opcional: seleccionar solo la columna de cierre ajustado
df = nvda['Close']
df.head()

C:\Users\hecto\AppData\Local\Temp\ipykernel_2696\4227794188.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nvda = yf.download(ticker, start="2010-01-01", end="2025-10-16", interval="1d")
[*********************100%***********************]  1 of 1 completed


Ticker,NVDA
Date,
2010-01-04,0.423830
2010-01-05,0.430019
2010-01-06,0.432770
2010-01-07,0.424289
2010-01-08,0.425205


In [5]:
# Función para crear ventanas
def create_dataset(series, window):
    X, y = [], []
    for i in range(len(series)-window):
        X.append(series[i:(i+window), 0])
        y.append(series[i+window, 0])
    return np.array(X), np.array(y)

window_size = 5
X, y = create_dataset(df[["NVDA"]].values, window_size)

# Separar en train y test (últimos 30 días)

n_test = 30
X_train, X_test = X[:-n_test], X[-n_test:]
y_train, y_test = y[:-n_test], y[-n_test:]

fechas_test = df.index[window_size:][-n_test:]

# Normalizar datos
scaler = MinMaxScaler()

# Para un único feature en ventanas, aplastamos para que scaler "vea" todos los puntos
# tanto de X_train como de y_train (opcional usar solo X_train; usar ambos suele ser útil si modelas el target escalado)
train_stack = np.vstack([X_train.reshape(-1,1), y_train.reshape(-1,1)])  # shape (-1,1)
scaler.fit(train_stack)

# 4) transformar X_train, X_test, y_train, y_test
X_train_scaled = scaler.transform(X_train.reshape(-1,1)).reshape(X_train.shape)
X_test_scaled  = scaler.transform(X_test.reshape(-1,1)).reshape(X_test.shape)
y_train_scaled = scaler.transform(y_train.reshape(-1,1)).reshape(-1)
y_test_scaled  = scaler.transform(y_test.reshape(-1,1)).reshape(-1)

In [6]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(window_size,)))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
# model.add(Dense(64, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation = "linear"))  # salida escalar

model.compile(optimizer="adam", loss="mse")
history = model.fit(X_train_scaled, y_train_scaled, epochs=30, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/30


C:\Users\hecto\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


111/111 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 4.0505e-04 - val_loss: 0.0013
Epoch 2/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4994e-05 - val_loss: 0.0011
Epoch 3/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5242e-05 - val_loss: 0.0011
Epoch 4/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5533e-05 - val_loss: 0.0011
Epoch 5/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5384e-05 - val_loss: 0.0012
Epoch 6/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5254e-05 - val_loss: 9.7940e-04
Epoch 7/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5199e-05 - val_loss: 0.0010
Epoch 8/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4933e-05 - val_loss: 8.6879e-04
Epoch 9/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4687e-05 - val_loss: 9.0027e-04
Epoch 10/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4459e-05 - val_loss: 8.9243e-04
Epoch 11/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4237e-05 - val_loss: 7.7827e-04

In [7]:
y_pred = model.predict(X_test_scaled)
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1,1))
y_test_rescaled = scaler.inverse_transform(y_test_scaled.reshape(-1,1))

# DataFrame comparativo
df_pred = pd.DataFrame({
    "Real": y_test_rescaled.flatten(),
    "Predicho": y_pred_rescaled.flatten()
}, index=fechas_test)

print(df_pred.head())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
                  Real    Predicho
Date                              
2025-09-04  171.650330  171.705551
2025-09-05  167.010590  171.780960
2025-09-08  168.300507  168.453369
2025-09-09  170.750366  168.069519
2025-09-10  177.320007  169.090515


In [8]:
df_pred.tail()

,Real,Predicho
Date,,
2025-10-09,192.570007,186.915863
2025-10-10,183.160004,189.383499
2025-10-13,188.320007,184.717636
2025-10-14,180.029999,186.438004
2025-10-15,179.830002,181.934860


In [9]:
X_scaled = scaler.transform(X.reshape(-1,1)).reshape(X.shape)
y_scaled = scaler.transform(y.reshape(-1,1)).reshape(y.shape)

In [10]:
y_pred_all = model.predict(X_scaled)  # predicciones en todo el set (train + test)
y_pred_all_rescaled = scaler.inverse_transform(y_pred_all.reshape(-1,1))
y_all_rescaled = scaler.inverse_transform(y.reshape(-1,1))

fechas_all = df.index[window_size:]  # fechas correspondientes

df_pred_all = pd.DataFrame({
    "Real": y_all_rescaled.flatten(),
    "Predicho": y_pred_all_rescaled.flatten()
}, index=fechas_all)



# Filtrar desde 2015
fecha_inicio = "2020-01-01"
df_filtrado = df.loc[fecha_inicio:]
df_pred_all_filtrado = df_pred_all.loc[fecha_inicio:]
df_pred_filtrado = df_pred.loc[fecha_inicio:]

fig = go.Figure()

# Serie completa real
fig.add_trace(go.Scatter(x=df_filtrado.index, y=df_filtrado["NVDA"],
                         mode="lines",
                         name="Serie completa (Real)",
                         line=dict(color="lightgray")))

# Predicciones de todo el modelo (train + test)
fig.add_trace(go.Scatter(x=df_pred_all_filtrado.index, y=df_pred_all_filtrado["Predicho"],
                         mode="lines",
                         name="Predicho (train + test)",
                         line=dict(color="orange")))

# Últimos 15 días - reales
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Real"],
                         mode="lines+markers",
                         name="Real (últimos 30 días)",
                         line=dict(color="blue")))

# Últimos 15 días - predichos
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Predicho"],
                         mode="lines+markers",
                         name="Predicho (últimos 30 días)",
                         line=dict(color="red", dash="dot")))

fig.update_layout(title="Predicción de NVDA con FNN",
                  xaxis_title="Fecha",
                  yaxis_title="NVDA 30 días",
                  legend=dict(x=0.01, y=0.99, bordercolor="black", borderwidth=1))

fig.show()

124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
